# Exploration stage

OPES-flooding simulations to harvest reactive pathways coupled with uncertainty-aware molecular dynamics simulations

Requirements:
* FLARE
* LAMMPS
* PLUMED

In [1]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [4]:
import numpy as np
import yaml
import sys
from pathlib import Path

from ase.visualize import view
from ase.io import write

sys.path.append("../../")

from mlputils.build import build_surface_FeCo, add_molecules

### Create initial configuration

$N_2$ molecule adsorbed on top of a FeCo(110) surface (or $2N$ for the products)

In [2]:
adsorbates = ['N2'] # or ['N','N']

folder = "./"+"-".join(adsorbates)+"/"
Path(folder).mkdir(exist_ok=True)

In [3]:
atoms = build_surface_FeCo(miller_index=(0,1,1),
                            layers=10, 
                            size=(4,3,1),
                            vacuum=15, 
                            fixed_layers=[0,2])

atoms = add_molecules(atoms, adsorbates, height=2., seed = 42 )

write(f'{folder}input.xyz',atoms,format='extxyz')

view(atoms,viewer='x3d')

ATOMS: 120 (48 fixed - 72 free)
Fixed atoms:  [  1   2   3   4  11  12  13  14  21  22  23  24  31  32  33  34  41  42
  43  44  51  52  53  54  61  62  63  64  71  72  73  74  81  82  83  84
  91  92  93  94 101 102 103 104 111 112 113 114]
Free atoms:  [  5   6   7   8   9  10  15  16  17  18  19  20  25  26  27  28  29  30
  35  36  37  38  39  40  45  46  47  48  49  50  55  56  57  58  59  60
  65  66  67  68  69  70  75  76  77  78  79  80  85  86  87  88  89  90
  95  96  97  98  99 100 105 106 107 108 109 110 115 116 117 118 119 120]


/leonardo/pub/userexternal/sperego0/envs/flare_v1.3.3/lib/python3.8/site-packages/ase/io/extxyz.py:311: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


### Generate FLARE input

We will generate the input starting from the configurations files stored in `configs/flare`. We will load one file per each of the four sections below. Note that there will be multiple options per each section, in the format: `section-XXX`.  

* **supercell**  : specify input configurations
* **flare_calc** : initialize the SGP or load from file (options: `init`,`file`)
* **dft_calc**   : use DFT calc (options: `qe`,`fake`)
* **otf**        : MD simulation and on-the-fly update settings (options: `lammps`,`ase`,`deal`)

In [4]:
configs_folder = '../../configs/flare/'

# load config files
settings = ['supercell',
            'flare_calc-file',      # load GP from file
            'dft_calc-qe',          # use QE as calculator
            'otf-lammps'            # use lammps for MD
            ]

config = {}
for section in settings:
    with open(f'{configs_folder}{section}.yaml') as file:
        config.update(yaml.load(file, Loader=yaml.FullLoader))

#### Initial config (supercell)

Default parameters: configs/flare/supercell.yaml

In [5]:
section = 'supercell'

config[section]['file'] = 'input.xyz'

#### Load calculator (flare_calc)

In [6]:
section = 'flare_calc'

config[section]['file'] = 'otf_flare.json'

# copy GP checkpoint the previous folder, e.g.:
# import shutil
# shutil.copy(f'../0_preliminary/{folder}/otf_flare.json',folder)

#### DFT Settings: QE (dft_calc)

In [7]:
section = 'dft_calc'

# If you want to change some option, e.g. 
#config[section]['kwargs']['input_data']['pseudo_dir'] = './'

#### OTF & MD settings: LAMMPS (otf)


Here we also add the PLUMED command inside LAMMPS

In [8]:
lmp_executable = "lmp" # path to lammps executable
temp = 700
seed = 1

timestep = 0.001
nsteps   = 10000

In [15]:
# Update parameters

section = 'otf'

config[section].update({ 'initial_velocity'  : temp,
                      'dt'                : timestep,
                      'number_of_steps'   : nsteps})
           
config[section]['md_kwargs'].update({'command' : lmp_executable,
                                     'group': [ f"free id {atoms.info['free']}",
                                                f"fixed id {atoms.info['fixed']}"],
                                                
                                     'fix'  : [ "1 all plumed plumedfile plumed.dat outfile p.log",
                                                "2 free nve",
                                                f"3 free temp/csvr {temp} {temp} 0.1 {seed}"] })

# Workaround for using OPES even when the GP restarts lammps before the first update of the bias
config[section]['md_kwargs']['shell'] = ['[ ! -s KERNELS ] && cp plumed-fresh.dat plumed.dat || cp plumed-restart.dat plumed.dat']

#### Write update input

In [12]:
# write input otf to file
with open(f'{folder}input-otf.yaml', 'w') as file:
    yaml.dump(config, file, sort_keys=False)

### Write PLUMED Input

We will use the template from `configs/plumed` and write the index via ASE

In [38]:
configs_folder = '../../configs/plumed/'

with open(f'{configs_folder}/plumed-flooding.dat', 'r') as file:
  plumed = file.read()

# Write atoms index in file
Fe_atoms  = np.argwhere ( atoms.get_atomic_numbers() == 26)[:,0]
Co_atoms  = np.argwhere ( atoms.get_atomic_numbers() == 27)[:,0]
N_atoms   = np.argwhere ( atoms.get_atomic_numbers() == 7)[:,0]
FeCo_atoms= list(Fe_atoms)+list(Co_atoms) 

plumed = plumed.replace('___TEMP___',str(temp))
plumed = plumed.replace('___Fe-ATOMS___',   ",".join( [str(i+1) for i in  Fe_atoms ]))
plumed = plumed.replace('___Co-ATOMS___',   ",".join( [str(i+1) for i in  Co_atoms ]) )
plumed = plumed.replace('___FeCo-ATOMS___', ",".join( [str(i+1) for i in  FeCo_atoms ]))
plumed = plumed.replace('___N-ATOMS___',    ",".join( [str(i+1) for i in  N_atoms ]))
plumed = plumed.replace('___N1-ATOM___',    ",".join( [str(i+1) for i in  [N_atoms[0]] ]))
plumed = plumed.replace('___N2-ATOM___',    ",".join( [str(i+1) for i in  [N_atoms[1]] ]))

# Create two files depending on whether to restart (append to file, but requires KERNELS to exist) or not

plumed_fresh = plumed.replace('___RESTART_MODE___','YES')
plumed_restart = plumed.replace('___RESTART_MODE___','YES').replace('#RESTART','RESTART')

with open(f'{folder}plumed-fresh.dat', 'w') as file:
  file.write(plumed_fresh)
with open(f'{folder}plumed-restart.dat', 'w') as file:
  file.write(plumed_restart)

print(plumed)

# Template file for PLUMED: Opes-flooding for N2 / FeCo

#RESTART

UNITS LENGTH=A

### GROUPS

N1:   GROUP ATOMS=120
N2:   GROUP ATOMS=121
N:    GROUP ATOMS=120,121
Fe:   GROUP ATOMS=0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118
Co:   GROUP ATOMS=1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119
FeCo: GROUP ATOMS=0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,55,57,59,61,63,65,67,69,71,73,75,77,79,81,83,85,87,89,91,93,95,97,99,101,103,105,107,109,111,113,115,117,119

### COLLECTIVE VARIABLES

# Coordination

### Perform on-the-fly training and MD

Run FLARE MD with the following command:

> flare-otf input-otf.yaml